In [97]:
from search import *
from games import *
import games

import copy

Felteszem hogy a két játékos nevei: **A** és **B**


Felteszem továbbá:
- A tábla egyik oldalának a hossza: *len* (ha 8x8 a tábla akkor len az 8), az indexelés azonban 0-7 ig megy, X és Y esetében is a nulladik index a tábla bal felső sarka
- *X* a vízszintes irányban a blokkok (tábla részei)
- *Y* a függőleges irányban a blokkok (tábla részei)

**A** vízszintesen rakhat
**B** függőlegesen rakhat


Egy lépés valid:
- **A** esetén a lerakható dominó kettő blokkja balról jobbra legyenek *1* és *2*:
    - Bármelyik sorban a *2*-es nem állhat $X_{len-len}$ indexen
    - Bármelyik sorban az *1*-es nem állhat $X_{len-1}$ indexen
    - Bármelyik sorban csak akkor rakható le máshova a dominó ha se *1*, se *2* nem ütközik **B** dominóival
    - Bármelyik sorban csak akkor rakható le dominó ha egymás mellett van kettő unobstructed hely


- **B** esetén a lerakható dominó kettő blokkja felülről lefelé legyenek *1* és *2*:
    - Bármelyik oszlopban a *2*-es nem állhat $Y_{len-len}$ indexen
    - Bármelyik oszlopban az *1*-es nem állhat $Y_{len-1}$ indexen
    - Bármelyik oszlopban csak akkor rakható le máshova a dominó ha se *1*, se *2* nem ütközik **A** dominóival
    - Bármelyik oszlopban csak akkor rakható le dominó ha egymás felett van kettő unobstructed hely

Itt amire szükség van:
- A tábla mérete változtatható legyen
- Tudni kell ki jön
- Tudni kell hogy a lépés valid-e
- Szükséges valahogyan reprezentálnom a táblát
- Szükséges egy cost function
  - Azaz ha **A**-nak ajánlok akkor azt a lépést válasszam ami a legrosszabb **B**-nek és vice versa

In [98]:
# nem térít vissza dictet csak módosítja az átadottat
def invalid_steps_remover_returns_new_dict(state, dict__): 
    
    for i in range(table_size):
        for j in range(table_size):
            if j < table_size-1 and ( state[i][j] != 0 or state[i][j+1] != 0 ) and (i, j) in dict__["A"]:
                dict__["A"].remove((i, j))
        
        
    for i in range(table_size):
        for j in range(table_size):
            if i < table_size-1 and ( state[i][j] != 0 or state[i+1][j] != 0 ) and (i, j) in dict__["B"]:
                dict__["B"].remove((i, j))
    

In [99]:
table_size = 5
dict_ = {}
dict_["A"] = []
dict_["B"] = []

In [100]:
# generál egy adott méretű táblát
def starter_kit(size):
    
    table = []
    z = []
    
    for i in range(size):
        z.append(0)
    for i in range(size):
        table.append(z.copy())
    return (table)

In [101]:
def initial_run(state):
    """ 
        feltölti a dict_-emet a lehetséges pozíciókkal az átadott tábla alapján
    """
    for i in range(table_size):
        for j in range(table_size):
            if j < table_size-1 and state[i][j] == 0 and state[i][j+1] == 0:
                #print( "i és j:" , i, "  " ,j)
                dict_["A"].append((i, j))
    
                    
    """print()
    print()"""
        
        
    for i in range(table_size):
        for j in range(table_size):
            if i < table_size-1 and state[i][j] == 0 and state[i+1][j] == 0:
                #print( "i és j:" , i, "  " ,j)
                dict_["B"].append((i, j))

A Game osztály hasonlít a probléma osztályra, de minden egyes state-re (állapot) van egy utility (hasznosság) függvénye a probléma osztályban lévő útvonalköltség és célteszt helyett. Hogy egy játékot létrehozzunk, létrehozunk egy alosztályt aminek átadjuk a Game osztályt, és implementáljuk a lehetséges akciókat, a hasznosságot, és a végtesztet.
A megjelenítést és a rákövetkező lépéseket lehetséges implementálni, vagy lehet az alapértelmezett metódusokból örököltetni őket. Szükséges még beállítani a .initial azaz a kezdő állást; ezt lehetséges a konstruktorban végrehajtani.

In [102]:
class StopGate(Game):
    def __init__(self, table_size, some_given_state, moves):
        self.initial = games.GameState(to_move = "A", utility = 0, board = copy.deepcopy(some_given_state), moves = copy.deepcopy(moves))
        
    def actions(self, state):
        if state.to_move == "A":
            return state.moves["A"]
        if state.to_move == "B":
            return state.moves["B"]
    
    
    def result(self, state, move):
        self.display(state)
                       
        # ha a vízszintesen rakó illető a soros
        if state.to_move == "A":
            
            board = copy.deepcopy(state.board)
            board[move[0]][move[1]] = "-"
            board[move[0]][move[1] + 1] = "-"
            
            A_movesLenBefore = len(state.moves["A"])
            B_movesLenBefore = len(state.moves["B"])
            
            
            moves = copy.deepcopy(state.moves)
            
            invalid_steps_remover_returns_new_dict(board, moves)
            
            
            A_movesLenAfter = len(moves["A"])
            B_movesLenAfter = len(moves["B"])
            
            return games.GameState(to_move = "B",
                                    utility = self.compute_utility("A", A_movesLenBefore, B_movesLenBefore,
                                                                   A_movesLenAfter, B_movesLenAfter),
                                    board = board, moves = moves)
        
        # ha a függőlegesen rakó illető a soros
        if state.to_move == "B":
            
            board = copy.deepcopy(state.board)
            board[move[0]][move[1]] = "|"
            board[move[0] + 1][move[1]] = "|"

            A_movesLenBefore = len(state.moves["A"])
            B_movesLenBefore = len(state.moves["B"])
            
            
            moves = copy.deepcopy(state.moves)
            
            invalid_steps_remover_returns_new_dict(board, moves)
            
            
            A_movesLenAfter = len(moves["A"])
            B_movesLenAfter = len(moves["B"])
            
            return games.GameState(to_move = "A",
                                    utility = self.compute_utility("B", A_movesLenBefore, B_movesLenBefore,
                                                                   A_movesLenAfter, B_movesLenAfter),
                                    board = board, moves = moves)
            
            
    
    # 
    def compute_utility(self, player, A_before, B_before, A_after, B_after):
        
        """
            
            A minmax alapelve hogy a max törekszik a lehető legnagyobb értékre
            A min törekszik a lehető legkisebb értékre
        
            ötlet, hogy az legyen a utility function, hogy az ellenfél lehetséges
            lépéseiből mennyit veszek el a saját lépésemmel

            ha így oldom meg ahogy leírom alább, akkor nem kell a "return state.utility if player=='A' else -state.utility"
            hanem csak szimplán a "return state.utility"
            
            maximizer:
            ha A lép, és amit lépett azzal megnyeri a játékot (azaz B lépéslistája üressé válik), az legyen  10
            ha A lép, és amit lépett azzal elveszti a játékot (azaz A lépéslistája üressé válik), az legyen -10
            
            minimizer:
            ha B lép, és amit lépett azzal megnyeri a játékot (azaz A lépéslistája üressé válik), az legyen -10
            ha B lép, és amit lépett azzal elveszti a játékot (azaz B lépéslistája üressé válik), az legyen  10
            
            egyébiránt:
            0
            
        """
        
        #(self, moves, player, A_before, B_before, A_after, B_after)
        
        if player == "A":
            if A_after == 0 and B_after != 0:
                return -10
            
            if B_after == 0:
                return  10
            
            else:
                return 0 
            
        
        if player == "B":
            if A_after != 0 and B_after == 0:
                return 10
            
            if A_after == 0:
                return  -10
            
            else:
                return 0 
            
        
        
    def utility(self, state, player):
        return state.utility 
        
    # meg kell adnia, hogy elértük-e a játék végét
    # kétféleképp lehet vége, vagy A, vagy B nem tud rakni
    def terminal_test(self, state):
        
        if not bool(state.moves["A"]):
            return True
        if not bool(state.moves["B"]):
            return True
        else: return False

        
    
    def display(self, state):
        for i in range(table_size):
            for j in range(table_size):
                print(state.board[i][j], end = " ")
            print()
        print()
    
    

In [103]:
table_size = 5
given_position = [["-", "-", "-", "-", "|"],
                  ["-", "-", 0, 0, "|"],
                  ["|", "|", 0, 0, 0],
                  ["|", "|", "|", 0, 0],
                  ["-", "-", "|", "-", "-"]]

table_size = 4
given_position = [["-", "-", 0, "|"],
                  [0, "-", "-", "|"],
                  [0, 0, 0, "|"],
                  [0, 0, 0, "|"]]

In [104]:
dict_["A"].clear()
dict_["B"].clear()

In [105]:
initial_run(given_position)

In [106]:
def invalid_steps_remover(state):
    """ 
        megkap egy state-et, azaz egy táblát, és ezen tábla alapján eltávolítja
        a globális dict_ ből azon állapotokat, amelyek a state alapján már illegálisak        
    """
    for i in range(table_size):
        for j in range(table_size):
            if j < table_size-1 and ( state[i][j] != 0 or state[i][j+1] != 0 ) and (i, j) in dict_["A"]:
                dict_["A"].remove((i, j))
        
        
    for i in range(table_size):
        for j in range(table_size):
            if i < table_size-1 and ( state[i][j] != 0 or state[i+1][j] != 0 ) and (i, j) in dict_["B"]:
                dict_["B"].remove((i, j))

In [107]:
dict_

{'A': [(1, 2), (2, 2), (2, 3), (3, 3)], 'B': [(1, 2), (1, 3), (2, 3), (2, 4)]}

In [108]:
invalid_steps_remover(given_position)

In [109]:
dict_

{'A': [(1, 2), (2, 2), (2, 3), (3, 3)], 'B': [(1, 2), (1, 3), (2, 3), (2, 4)]}

In [110]:
sg = StopGate(table_size = table_size, some_given_state = given_position, moves = dict_)

In [111]:
print(games.minmax_decision(sg.initial, sg))

- - - - | 
- - 0 0 | 
| | 0 0 0 
| | | 0 0 
- - | - - 

- - - - | 
- - - - | 
| | 0 0 0 
| | | 0 0 
- - | - - 

- - - - | 
- - - - | 
| | 0 0 0 
| | | 0 0 
- - | - - 

- - - - | 
- - - - | 
| | 0 0 | 
| | | 0 | 
- - | - - 

- - - - | 
- - 0 0 | 
| | 0 0 0 
| | | 0 0 
- - | - - 

- - - - | 
- - 0 0 | 
| | - - 0 
| | | 0 0 
- - | - - 

- - - - | 
- - 0 0 | 
| | 0 0 0 
| | | 0 0 
- - | - - 

- - - - | 
- - 0 0 | 
| | 0 - - 
| | | 0 0 
- - | - - 

- - - - | 
- - 0 0 | 
| | 0 0 0 
| | | 0 0 
- - | - - 

- - - - | 
- - 0 0 | 
| | 0 0 0 
| | | - - 
- - | - - 

- - - - | 
- - | 0 | 
| | | 0 0 
| | | - - 
- - | - - 

- - - - | 
- - 0 0 | 
| | 0 0 0 
| | | - - 
- - | - - 

(2, 2)
